<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao22_FastTextLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abordagem 1: Classificação com FastText (Embeddings) e LSTM

Este notebook demonstra como usar o **FastText** como um gerador de *embeddings* (vetores de palavras) que são então usados para alimentar uma rede LSTM no Keras.

A principal vantagem do FastText sobre o Word2Vec é o uso de **n-gramas de caracteres** (informação de subpalavras). Isso permite que ele crie vetores para palavras que não estavam no vocabulário de treino (OOV - Out-of-Vocabulary).

O fluxo é quase idêntico ao do notebook Word2Vec:
1. Carregar dados.
2. Treinar um modelo FastText (via `gensim`) nos textos.
3. Preparar dados para o Keras (Tokenizer, Padding).
4. Criar a Matriz de Embedding usando os vetores do FastText.
5. Construir, treinar e avaliar o modelo LSTM.

In [1]:
# Célula 1: Instalação e Importações
!pip install gensim

import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import FastText # <-- A MUDANÇA ESTÁ AQUI
import re
import warnings

warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 34.1 MB/s eta 0:00:00


## Passo 1: Carregar e Preparar Dados

In [2]:
# Célula 2: Carregar os dados
categorias = ['comp.graphics', 'sci.crypt']
dados = fetch_20newsgroups(subset='all', categories=categorias, shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))
X_train_text, X_test_text, y_train, y_test = train_test_split(dados.data, dados.target, test_size=0.2, random_state=42)

print(f"Amostras de treino: {len(X_train_text)}")
print(f"Amostras de teste: {len(X_test_text)}")

Amostras de treino: 1571
Amostras de teste: 393


## Passo 2: Treinar o Modelo FastText (Gensim)

O processo é o mesmo do Word2Vec, mas trocamos a classe `Word2Vec` por `FastText`.

In [3]:
# Célula 3: Pré-processamento para o Gensim
def preprocess_text_gensim(text):
    text = re.sub(r'\W+', ' ', text) # Remove caracteres não-alfanuméricos
    text = text.lower()
    return text.split()

textos_completos = X_train_text + X_test_text
textos_tokenizados = [preprocess_text_gensim(doc) for doc in textos_completos]

print(f"Primeiro documento tokenizado (para Gensim):\n{textos_tokenizados[0][:20]}...")

Primeiro documento tokenizado (para Gensim):
[]...


In [4]:
# Célula 4: Treinar o modelo FastText

DIM_EMBEDDING = 100
WINDOW_SIZE = 5
MIN_COUNT = 3
WORKERS = 4

print("Treinando modelo FastText...")
ft_model = FastText( # <-- A MUDANÇA ESTÁ AQUI
    sentences=textos_tokenizados,
    vector_size=DIM_EMBEDDING,
    window=WINDOW_SIZE,
    min_count=MIN_COUNT,
    workers=WORKERS,
    min_n=3, # Tamanho mínimo dos n-gramas de caracteres
    max_n=6  # Tamanho máximo dos n-gramas de caracteres
)

print("Treino do FastText concluído.")
vocab_size_ft = len(ft_model.wv.key_to_index)
print(f"Tamanho do vocabulário do FastText (com min_count={MIN_COUNT}): {vocab_size_ft} palavras")

# A grande vantagem: gerar vetor para palavra OOV
palavra_teste_oov = 'ultramegasupercomputador'
if palavra_teste_oov not in ft_model.wv:
    print(f"'{palavra_teste_oov}' NÃO está no vocabulário treinado.")
    # Mas ainda podemos obter um vetor!
    vetor_oov = ft_model.wv[palavra_teste_oov]
    print(f"Mas o FastText gerou um vetor para ela: {vetor_oov.shape}")

Treinando modelo FastText...
Treino do FastText concluído.
Tamanho do vocabulário do FastText (com min_count=3): 9871 palavras


## Passo 3 & 4: Preparar Keras e Matriz de Embedding

In [5]:
# Célula 5: Tokenização e Padding do Keras
MAX_PALAVRAS_VOCAB = 10000
MAX_COMPRIMENTO_SEQ = 250

tokenizer = Tokenizer(num_words=MAX_PALAVRAS_VOCAB, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text)

X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_test_seq = tokenizer.texts_to_sequences(X_test_text)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_COMPRIMENTO_SEQ, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_COMPRIMENTO_SEQ, padding='post', truncating='post')

word_index = tokenizer.word_index
vocab_size_keras = min(MAX_PALAVRAS_VOCAB, len(word_index) + 1)

print(f"Tamanho do vocabulário do Keras: {vocab_size_keras}")

Tamanho do vocabulário do Keras: 10000


In [7]:
# Célula 6: Construção da Matriz de Embedding (usando ft_model)

embedding_matrix = np.zeros((vocab_size_keras, DIM_EMBEDDING))
palavras_encontradas = 0
palavras_nao_encontradas = 0

for word, i in word_index.items():
    if i >= MAX_PALAVRAS_VOCAB:
        continue

    # Usamos o ft_model.wv. A vantagem é que mesmo que 'word'
    # não esteja no vocabulário treinado (por min_count),
    # o FastText pode inferir um vetor para ela.
    try:
        embedding_matrix[i] = ft_model.wv[word]
        palavras_encontradas += 1
    except KeyError:
        # No FastText (gensim >= 4.0), isso é raro, mas tratamos por segurança
        embedding_matrix[i] = np.zeros(DIM_EMBEDDING)
        palavras_nao_encontradas += 1

print(f"Matriz de Embedding criada com formato: {embedding_matrix.shape}")
print(f"{palavras_encontradas} palavras do Keras mapeadas.")
print(f"{palavras_nao_encontradas} palavras não mapeadas (improvável com FastText).")

Matriz de Embedding criada com formato: (10000, 100)
9999 palavras do Keras mapeadas.
0 palavras não mapeadas (improvável com FastText).


## Passo 5 & 6: Construir e Treinar o Modelo LSTM

In [10]:
# Célula 7: Definindo o modelo LSTM
modelo_ft_lstm = Sequential(name="Modelo_FastText_LSTM")

modelo_ft_lstm.add(Embedding(
    input_dim=vocab_size_keras,
    output_dim=DIM_EMBEDDING,
    input_length=MAX_COMPRIMENTO_SEQ,
    weights=[embedding_matrix],
    trainable=False # Congelar a camada
))

modelo_ft_lstm.add(SpatialDropout1D(0.2))
modelo_ft_lstm.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
modelo_ft_lstm.add(Dense(16, activation='relu'))
modelo_ft_lstm.add(Dense(1, activation='sigmoid'))

modelo_ft_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelo_ft_lstm.summary()

Model: "Modelo_FastText_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_1             │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,000,000 (3.81 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,000,000 (3.81 MB)

In [11]:
# Célula 8: Treinamento e Avaliação
y_train_np = np.array(y_train)
y_test_np = np.array(y_test)

print("\nTreinando o modelo FT+LSTM...")

historico = modelo_ft_lstm.fit(
    X_train_pad,
    y_train_np,
    epochs=100,
    batch_size=32,
    validation_data=(X_test_pad, y_test_np),
    verbose=2
)

loss, acuracia_ft_lstm = modelo_ft_lstm.evaluate(X_test_pad, y_test_np, verbose=0)

print(f"\nAcurácia (FastText Embeddings + LSTM): {acuracia_ft_lstm:.4f}")


Treinando o modelo FT+LSTM...
Epoch 1/100
50/50 - 18s - 354ms/step - accuracy: 0.5188 - loss: 0.6860 - val_accuracy: 0.5471 - val_loss: 0.6789
Epoch 2/100
50/50 - 14s - 278ms/step - accuracy: 0.5398 - loss: 0.6787 - val_accuracy: 0.5547 - val_loss: 0.6766
Epoch 3/100
50/50 - 19s - 389ms/step - accuracy: 0.6416 - loss: 0.6491 - val_accuracy: 0.5598 - val_loss: 0.7477
Epoch 4/100
50/50 - 13s - 263ms/step - accuracy: 0.5773 - loss: 0.6640 - val_accuracy: 0.5369 - val_loss: 0.6758
Epoch 5/100
50/50 - 20s - 399ms/step - accuracy: 0.5856 - loss: 0.6424 - val_accuracy: 0.6921 - val_loss: 0.5926
Epoch 6/100
50/50 - 21s - 423ms/step - accuracy: 0.6620 - loss: 0.6276 - val_accuracy: 0.7023 - val_loss: 0.5669
Epoch 7/100
50/50 - 13s - 255ms/step - accuracy: 0.7015 - loss: 0.6061 - val_accuracy: 0.7557 - val_loss: 0.5460
Epoch 8/100
50/50 - 13s - 257ms/step - accuracy: 0.7377 - loss: 0.5672 - val_accuracy: 0.7863 - val_loss: 0.5206
Epoch 9/100
50/50 - 13s - 257ms/step - accuracy: 0.6875 - loss: 0